### need to review some missing dependencies - or re-write this section (NOW WE PROCESS THE HLA TYPES TO MATCH WITH OUR AVAILABLE FUSION FILES - patient specific matching)

In [ ]:
#the final output file for each cohort here were included in the handover folder
#I may have removed one dependent file - "pvf-list" 
#.....................................................................................
#TCGA - WES x 1058 <-> RNAseq x 1095. should expect 1055 rows)

##download all raw files from S3 - renamed as raw_output/${ID}_final.result.txt
# location of hla typing result from WES matched normal: 
# s3://crm.tumorstudy.analysis/tcga-run/HLA_Typing/HLA-HD/WES_slicedBAM/TCGA-*_final.result.txt
# did not use result from RNAseq tumour / normal (n=40) this round!
##handle "not_typed" by replacing space with underscore > extract col 2 & 3 > exclude "-" > 
##convert 2 cols into 1 col > extract 4digits only = clean file (should only have 1055 in total by the end)

NUM=$1
ID=$(sed -n ${NUM}p ../FINAL_genomicBAM_UUID_fileID_fileName.txt | awk '{print $1}')

if [[ -e raw_output/${ID}_final.result.txt ]]
then
    sed 's/ /_/g' raw_output/${ID}_final.result.txt | sed 's/Not_typed/-/g' | awk '{print $2,$3}' | sed 's/ /\n/g' | grep -v "^-" | sed 's/\(\*[0-9]\{2\}:[0-9]\{2\}\)\(:[0-9]\{2\}\)/\1/g' | uniq >> clean_output/${ID}_clean.result.txt
    all_alleles=$(while read gene; do grep $gene clean_output/${ID}_clean.result.txt; done < pvf-list | sed 's/HLA-D/D/g' | paste -sd',')
    echo -e "${ID}\t${all_alleles}" >> ../TCGA_patient-specific-pvfAlleles.txt
fi

#.....................................................................................
#MyBrCa - 990-123=867

##download result file from S3 - renamed to raw_output/hlahd_${ID}N.txt
# location of hla typing result: 
# s3://crm.tumorstudy.analysis/mybrca-run/HLA_Typing/HLA-HD/WES/BAM/*N_bam/result/*N_WES-MHC-bam_final.result.txt
# did not use HLA-HD output from RNAseq tumour (n=123) this round! 
##format them similarly to the TCGA ones

NUM=$1
ID=$(sed -n ${NUM}p MYBRCA-867HLAHD-IDs.txt)

sed 's/ /_/g' raw_output/hlahd_${ID}N.txt | sed 's/Not_typed/-/g' | awk '{print $2,$3}' | sed 's/ /\n/g' | grep -v "^-" | sed 's/\(\*[0-9]\{2\}:[0-9]\{2\}\)\(:[0-9]\{2\}\)/\1/g' | uniq >> clean_output/hlahd_${ID}N_clean.txt
all_alleles=$(while read gene; do grep $gene clean_output/hlahd_${ID}N_clean.txt; done < pvf-list | sed 's/HLA-D/D/g' | paste -sd',')
echo -e "${ID}\t${all_alleles}" >> ../MYBRCA_patient-specific-pvfAlleles.txt

#.....................................................................................
#NTUH - for the 223 samples
##format NTUH sent us de list first
#results are stored on Dropbox : BRCA_NTUH_CRMY\NTUH\HLA\neopred_TW.HLA.summary.normal.selected.allhlatypes.txt
NUM=$1
ID=$(sed -n ${NUM}p NTUH_IDs.txt)
grep $ID neopred_TW.HLA.summary.normal.selected.allhlatypes.txt | awk '{print $3,$4}' >> ntuh_output/hlahd_${ID}.txt

##extract only the first two lines of each hla gene
NUM=$1
ID=$(sed -n ${NUM}p NTUH_IDs_inconsistent.txt)

while read gene; do
grep -m 2 $gene ntuh-original/hlahd_${ID}.txt >> ntuh_output/hlahd_${ID}.txt
done < ntuh-hlahd-genes

##now extract only the relevant genes from each file, exclude Not typed, only include up to 4 digits, only include Dxx*xx:xx instead of HLA-Dxx*xx:xx, REMOVED DUPS!!
NUM=$1
ID=$(sed -n ${NUM}p NTUH_IDs.txt)

while read gene; do
grep $gene ntuh_output/hlahd_${ID}.txt | awk '{print $NF}' >> ntuh_output_clean/hlahd_${ID}_clean.txt #this should give exactly 40 lines - more of a sanity check
done < pvf-list

all_alleles=$(grep -v Not ntuh_output_clean/hlahd_${ID}_clean.txt | uniq | paste -sd',' | sed 's/,HLA-D/,D/g' | sed 's/\(\*[0-9]\{2\}:[0-9]\{2\}\)\(:[0-9]\{2\}\)/\1/g')

echo -e "${ID}\t${all_alleles}" >> NTUH_patient-specific-pvfAlleles.txt

##upload to $S3/tmp/NTUH_patient-specific-pvfAlleles.txt

### (NOW WE RUN NEOANTIGEN PREDICTION using patient specific HLA type that we cleaned up in the previous step)

In [ ]:
#we will run pvf based on the IDs available. Write it in a way that handles missing fusion files/ wrong ID / wrong cohort entered
#remember to manually conda activate agf-condaenv in screen and remove docker container esp before rerun
#(only need conda env on main server :D - stereoseq no need)

workdir="/shared/minhui/PVACFUSE"
cohort=$1
NUM=$2
datadir=""
s3dir=""

ID=$(sed -n ${NUM}p ${cohort}_patient-specific-pvfAlleles.txt | awk '{print $1}')
ALLELES=$(sed -n ${NUM}p ${cohort}_patient-specific-pvfAlleles.txt | awk '{print $2}')

#extract ID for getting the native fusion (filtered) file.
#handle NTUH ID differently due to it's inconsistent naming- will use the first matching name (specifically for handling sample A010087L and A010087R. will use L for this round of analysis
if [[ $cohort == "NTUH" ]]; then
    ID_long=$(grep -m 1 $ID /shared/minhui/analysis/${cohort}/NTUH_ID_223only.txt)
elif [[ $cohort == "TCGA" || $cohort == "MyBrCa" ]]; then
    ID_long=$ID
else
    echo "PLEASE ONLY ENTER NTUH / TCGA / MyBrCa as your first command line argument. Run terminated $(date | awk '{print $3"/"$2"/"$NF"_"$4}')" >> ${workdir}/${cohort}_agfPVF.failed.txt
fi

for tool in Arriba FusionCatcher; do
	if [[ $tool == "Arriba" ]]; then 
		toool="arr"
		agf_tool="arriba"
	elif [[ $tool == "FusionCatcher" ]]; then
		toool="fc"
		agf_tool="fusioncatcher"
	fi
done

	datadir="/shared/minhui/analysis/${cohort}/${tool}/native-fusion-files-filtered"

	if [[ -e ${datadir}/${toool}_${ID_long}_filtered.tsv ]]; then
	
		if [[ ! -d ${workdir}/${cohort}/sample_${ID} ]]; then
        		mkdir ${workdir}/${cohort}/sample_${ID}
		fi
		
		echo ""LINE_${NUM} SAMPLE_${ID} ${tool}: START $(date | awk '{print $3"/"$2"/"$NF"_"$4}')" >> ${workdir}/${cohort}_agfPVF.log.txt

		#run agf first
        	echo -e "\n\n ------------ AGFusion ------------ " >> ${workdir}/${cohort}/sample_${ID}/sample_${ID}_${toool}.log.txt 2>&1
        	echo -e "$(date | awk '{print $3"/"$2"/"$NF"_"$4}')\n\n" >> ${workdir}/${cohort}/sample_${ID}/sample_${ID}_${toool}.log.txt 2>&1
        	agfusion batch \
        	-f ${datadir}/${toool}_${ID_long}_filtered.tsv \
        	-a ${agf_tool} \
        	-db /stereoseq/minhui/ref/agfusion.homo_sapiens.95.db \
        	-o ${workdir}/${cohort}/sample_${ID}/agf_sample_${ID}_${toool} \
        	--middlestar \
        	--noncanonical >> ${workdir}/${cohort}/sample_${ID}/sample_${ID}_${toool}.log.txt 2>&1

        	#THEN PVF
        	echo -e "\n\n ------------ pVacFuse ------------ " >> ${workdir}/${cohort}/sample_${ID}/sample_${ID}_${toool}.log.txt 2>&1
        	echo -e "$(date | awk '{print $3"/"$2"/"$NF"_"$4}')\n\n" >> ${workdir}/${cohort}/sample_${ID}/sample_${ID}_${toool}.log.txt 2>&1
        	docker run -v ${workdir}/${cohort}/sample_${ID}:/home/MY_STUFF/ griffithlab/pvactools:4.0.4 pvacfuse run \
        	-a sample_name -e1 8,9,10 -e2 15 --iedb-install-directory /opt/iedb -k -t 8 \
        	/home/MY_STUFF/agf_sample_${ID}_${toool} pvf_sample_${ID}_${toool} \
        	${ALLELES} NetMHCpan NetMHCIIpan /home/MY_STUFF/pvf_sample_${ID}_${toool} >> ${workdir}/${cohort}/sample_${ID}/sample_${ID}_${toool}.log.txt 2>&1

		echo ""LINE_${NUM} SAMPLE_${ID} ${tool}: DONE  $(date | awk '{print $3"/"$2"/"$NF"_"$4}')" >> ${workdir}/${cohort}_agfPVF.log.txt

	else
		echo "NATIVE FUSION FILE FOR ${cohort} ${ID_long} ${tool} unavailable. Run terminated $(date | awk '{print $3"/"$2"/"$NF"_"$4}')" >> ${workdir}/${cohort}_agfPVF.failed.txt
	fi
done

s3dir="s3://crm.tumorstudy.analysis/NTUHxCRMY/${cohort,,}-run/SampleSpecificNeoantigenPrediction"
aws s3 cp ${workdir}/${cohort}/sample_${ID}/ ${s3dir}/sample_${ID}/ --recursive

### (NOW WE ANALYSE THE FUSIONS WHILE NEOANTIGEN PREDICTION IS RUNNING)